# Getting started with python API for Google Earth Engine (GEE)

## What is GEE and why use it?
(quote from the official website)
____________________________________________________________________________________________
Google Earth Engine is a geospatial processing service. With Earth Engine, you can perform geospatial processing at scale, powered by Google Cloud Platform. The purpose of Earth Engine is to:

    * Provide an interactive platform for geospatial algorithm development at scale
    * Enable high-impact, data-driven science
    * Make substantive progress on global challenges that involve large geospatial datasets
____________________________________________________________________________________________

More information on Google Earth Engine can be found here: https://developers.google.com/earth-engine

### A couple of important points are summarized below: 

__-->__ GEE hosts a ***multi-petabyte catalog of satellite imagery and other geospatial datasets*** on their servers ***(1 petabyte = 1024 terabytes, or a million gigabytes)*`. More information can be found here: https://developers.google.com/earth-engine/datasets. 

__-->__ GEE uses Java script API but also has a python API. Both APIs aaccess the same server-side functionality. However, there are syntax differences between the two programming langauges. More information and examples can be found here: https://developers.google.com/earth-engine/guides/python_install#syntax




# Today's exercise: Case study - Hurricane Maria (2017) impact on Puerto Rico 


_**IMPORTANT:**_ This script ported from Google Earth Engine (GEE) Java script and adaptaed to python jupyter notebook.

_**--> Source & citation provided below.**_

_**Source of original GEE script:**_ NASA-ARSET-training - Satellite Observations for Analyzing Natural Hazards on Small Island Nations
https://appliedsciences.nasa.gov/join-mission/training/english/arset-satellite-observations-analyzing-natural-hazards-small-island

_**Full citation:**_
Podest, E.; McCartney, S.; Mehta, A.; Englander, J.; Hamlington, B.; Stanley, T. (2021). Satellite Observations for Analyzing Natural Hazards on Small Island Nations. NASA Applied Remote Sensing Training Program (ARSET). https://appliedsciences.nasa.gov/join-mission/training/english/arset-satellite-observations-analyzing-natural-hazards-small-island



_**--> Revisions and edits by Atef AMRICHE, Jeff Liu 2022.**_

**-----------------------------------------------------------------------------------**

# VERY IMPORTANT:
____________________________________________________________________________________________
____________________________________________________________________________________________
General information is formatted with an arrow at the beginning. 

**--> general information** about a certain tool or library or dataset ... and link provided at the end where available.


python script analysis steps are presented in bullet points. Important information is highlighted in **bold** and code is notated in `preformatted text`. example below:
- step 1 python analysis: import study area **"name of study area"** shapefile.
to do this we use the command `example python command`. Other important information about this command or type of command can be found here (*example link*)
- step 2 python analysis: use the command `example python command` to do the following operation.
____________________________________________________________________________________________
____________________________________________________________________________________________

# ... Now let's begin !!

## The following libraries are imported to use in this demo.

- `ee` is the earth-engine python library that is installed & authenticated uzing a personal/professional Google Earth Engine account before use. More information can be found here: https://developers.google.com/earth-engine/guides/python_install


- `geemap` is a python package that allows for interactive use of Google Earth Engine mapping and data analysis functions. More information can be found here: https://geemap.org/

    The specific function `geemap` allows for creating interactive GEE maps that have a __base layer__ (google maps / terrain / other) plus a __specific result(s)__ overlayed on top (ex: satellite image, boundary of the study area, etc.)

In [9]:
# import necessary libraries
import ee
import geemap

## Authentication

### Step 1: Google Account and Cloud Project

Before you begin, let's have your Google Earth Engine Cloud Project created.

- Navigate to: https://console.cloud.google.com/projectcreate
    
    Do the following:

    <img src="notebook_images/image009.png" width="800"  />

- Click `CREATE` if you find everything good! You should be able to see the project dashboard as follows:

    <img src="notebook_images/image011.png" width="800"  />

- Now we need to enable Google Earth Engine API for the project that we created in order to access its services.

    <img src="notebook_images/image013.png" width="800"  />

    <img src="notebook_images/image015.png" width="800"  />
    
    <img src="notebook_images/image017.png" width="800"  />

    <img src="notebook_images/image019.png" width="800"  />

### Step 2: Token Generation and Project Authentication

Run the code block below:

In [10]:
# you only need to run the first time running the notebook, or if you haven't run it in more than a week
# comment it out once you've run it the first time
ee.Authenticate()


True

This should automatically open your browser and open up the Notebook Authenticator page

If it is not, navigate to this page: https://code.earthengine.google.com/client-auth

Do the following:

<img src="notebook_images/image021.png" width="800"  />


Note that you might see warning suggesting you should register the project for GEE. If so, do the following:

<img src="notebook_images/image023.png" width="800"  />

<img src="notebook_images/image025.png" width="800"  />

<img src="notebook_images/image027.png" width="800"  />

- Once you've finished registering the project, go back to https://code.earthengine.google.com/client-auth and hit "GENERATE TOKEN"
- Copy the token on the page and paste the token on the console (or any pop-up window that you see that requires this token)
- Hit enter and you're ready to go

### Step 3: Initialize

In [11]:
# initialize earth engine
ee.Initialize()

- __User input - event start and end dates -__ Here, we specify a date interval to search for satellite images before and after the hurricane. This must be an interval because satellite images are __NOT__ necessarily acquired on the same dates each year.

In [12]:
# Set dates for pre- and post-hurricane event
pre_event_beg = '2017-07-15'
pre_event_final = '2017-07-31'

post_event_beg = '2017-09-19'
post_event_final = '2017-09-25'

## Load the polygon for the study area "Puerto Rico"

__-->__ ***The FAO GAUL*** (Food and Agriculture Organization - Global Administrative Unit Layers) simplified 500m 2015 is a ***geospatial dataset*** that is part of the GEE collection. More information can be found here: https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0?hl=en

This dataset contains country boundaries as well as other information. It can be loaded using the command `ee.FeatureCollection`. Then, we utilize the filtering function to examine the column `ADM0_NAME`, which contains the country name. We apply the GEE filter `Equal` using the command `ee.Filter.eq` to extract the polygon of our study area **"Puerto Rico"**. 

More information on GEE filters and how to use them can be found here: https://developers.google.com/s/results/earth-engine?q=ee.filter

The specific information about the GEE filter "equal" and how to use it can be found here: https://developers.google.com/earth-engine/apidocs/ee-filter-eq?hl=en




In [13]:
# Load the shapefile for your area of interest
roi = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Puerto Rico'))


## Load a map and add our results
Here, we specify the center of the map coordinates and the zoom level. This would vary depending on the study area location and size, and can be determined through trial and error. We use the command `geemap.Map` with the default Google maps as a base layer.
    - For other locations, first determine the center coordinates, then
    - Zoom less (smaller zoom number) for large areas, or zoom more (larger zoom number) for small areas

Following that, we use the command `Map.addLayer` to display the `"roi"` layer , which corresponds to the polygon of Puerto Rico created in the previous step. This command allows for the selection of several display options. In our case, we only used the 'color' to assign the gray color to the polygon of our study area. More information about this command can be found here: https://developers.google.com/earth-engine/apidocs/map-addlayer?hl=en

At the end, we simply use the command `Map` to actually display our output map in a separate window below the python code cell. Some general information about interactive maps and map formatting is provided below as it is used repeatedly throughout this exercise.


__-->__ Each time we want to generate a map output, we first use the command `Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)`. This would create a new map with the default Google maps base layer showing our study area. Then we use the command `Map.addLayer` to specify which layers (results) to display and what display parameters are used (color, minimum and maximum values, etc). Finally we use the command `Map` to actually display our output map below the python code cell. 

__-->__ In each map window, we can zoom in and out as it is an interactive map. 

__-->__ Multiple layers or results can be displayed in a single map. In the **top right corner**, we see an icon showing multiple squares on top of each other (representing layers). We can hover over that icon to view and toggle which layers to display.

__-->__ There are other buttons available on the left side of the map. Most of these we will not use in this exercise. They are:

    The first two buttons are +/- to zoom in and out. The third button is to go **full screen**. 
    We can press `esc` to exit full screen.

    The next five buttons **(not used in this exercise)** are to draw various shapes 
    (line, polygon, square/rectangle, circle, point, circle marker. 

    The next two buttons allow for **editing** and **deleting** existing shapes.

    The last button is a map search button (similar to google maps search). 

In [41]:

# Use Google maps as the default basemap & display the study area polygon on it.
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(roi, {'color': 'gray'}, 'Study Area')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

If all your commands runs normally, you should be able to see a map on the output! Note that the map is interactive, try playing with the map and explore our data by doing the following:

- Drag and pan
- Scroll to zoom in/out
- Explore the layers

    <img src="notebook_images/image029.png" width="800"  />

    <img src="notebook_images/image031.png" width="800"  />


### OPTIONAL - Convert Feature-Collection to GeoPandas-Dataframe
This can be done using the command ` geemap.ee_to_geopandas`  (example results shown below for the `roi`  layer)


__-->__ ***Using the geemap library*** - This offers may conversion commands to and other functionality to convert data between GEE variable types to common python geospatial and other variable types. Useful information can be found here https://geemap.org/common/?h=ee_to_#geemap.common.ee_num_round

In [15]:
# gpd_dataframe = geemap.ee_to_geopandas(roi, selectors=None, verbose=False)

# deprecated function, use the following instead
gpd_dataframe = geemap.ee_to_df(roi)

## Load SAR images from the European satellite Sentinel-1.

Notes:

- What is Remote Sensing?

 	https://www.earthdata.nasa.gov/learn/backgrounders/remote-sensing

- What is SAR? (including what is VV, VH band)

 	https://www.earthdata.nasa.gov/learn/backgrounders/what-is-sar

 	https://developers.google.com/earth-engine/tutorials/community/sar-basics
    
- What is Sentiel-1 satellite?

 	https://sentiwiki.copernicus.eu/web/s1-mission



We do this separately before the event and then after the event for our study area. 

__-->__ ***The COPERNICUS/S1_GRD*** are ***satellite images from Sentinel-1*** that are part of the GEE collection. More information can be found here: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD?hl=en

This dataset contains historical satellite images since 2014 (ongoing satellite mission). It can be loaded using the command `ee.ImageCollection`. Then, we apply multiple filters using the command `.filter`  to select a specific SAR product, orbit, resolution, boundary, dates, and bands. More specifically:

    The command "ee.Filter.eq" allows us to extract the "instrument mode" equal to "IW" (Interferometric Wide-swath)
    The command "ee.Filter.eq" allows us to extract the "orbit" equal to "Ascending"
    
    The command ".filterMetadata" allows us to extract the "spatial resolution" equal to "10m"
    
    The command ".filterBounds" allows us to clip the image to our "roi" study area

    The command ".filterDate" allows us to extract available satellite images between a start anad end data.
    Here, we used the dates we defined above to extract available images before and after the hurricane.
    
    The command ".select" allows us to extract the "specific bands (VV, VH)" for this SAR product
    
There is a large number of filter/selection commands in GEE. More information on these commands and how to use them can be found here: https://developers.google.com/s/results/earth-engine?q=ee.filter

A given study area is not necessarily covered by a single SAR image. Therefore, after extracting the available SAR images before and after the hurricane, we need to generate a single image mosaic covering our entire study area. For this, we use the command `.mosaic` and apply it to our Sentinel-1 collections before and after the hurricane. We use the command `.Clip` agaain to make have the final mosaic image clipped to our study area.   

In [16]:
# Load Sentinel-1 C-band SAR Ground Range collection (log scale): pre-event
S1collection_1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
                   .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                   .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
                   .filterMetadata('resolution_meters', 'equals' , 10) \
                   .filterBounds(roi) \
                   .filterDate(pre_event_beg, pre_event_final) \
                   .select('VV', 'VH')

# Load Sentinel-1 C-band SAR Ground Range collection (log scale): post-event
S1collection_2 = ee.ImageCollection('COPERNICUS/S1_GRD') \
                   .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                   .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
                   .filterMetadata('resolution_meters', 'equals' , 10) \
                   .filterBounds(roi) \
                   .filterDate(post_event_beg, post_event_final) \
                   .select('VV', 'VH')

# Create a mosaic for each collection
S1_1 = S1collection_1.mosaic().clip(roi)
S1_2 = S1collection_2.mosaic().clip(roi)

## Preprocess the SAR images
This step is common for SAR images. Basically, we apply a smoothing filter because SAR signal is grainy. In this case, we use the command `.focal_mean`  which calculates the average values of pixels within a certain radius **(in our case 30m)** and assign that value to the center pixel.

### Load a map and add our results 

First, we specify the center of the map coordinates and the zoom level (same step as before).
We use the command `geemap.Map` with the default Google maps as a base layer. 

Then, we add our SAR images (4 separate layers) using the command ".addLayer" four times. 
- 1st argument represents the variable we want to add (ex: S1_1 for Sentinel-1 collection 1)
- 2nd argument represents the visualization parameters (ex: minimum value = -25, maximum value = -5)
The min/max values change from one image to another, so we choose fixed values for consistent colors.
- 3rd argument is the name of the layer to be displayed on the map window.


In [17]:
# Apply speckle filter
SMOOTHING_RADIUS = 30
pre_event_filtered = S1_1.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters')
post_event_filtered = S1_2.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters')

# Add images to "Layers" in order to visualize them
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(S1_1, {'min':-25, 'max':-5}, 'S1-1')
Map.addLayer(S1_2, {'min':-25, 'max':-5}, 'S1-2')
Map.addLayer(pre_event_filtered, {'min':-25, 'max':-5}, 'S1-1-Filt')
Map.addLayer(post_event_filtered, {'min':-25, 'max':-5}, 'S1-2-Filt')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Combine the SAR image bands to generate a 3-band image for visualization
Here, we use the command `.cat` (short for concatenate) to combine the same SAR band (filtered VV or VH) and generate a 3-band image and visualize it in RGB colors. We use the command `.select` (same as before) to choose a specific band.


### Load a map and add our results

First, we specify the center of the map coordinates and the zoom level (same step as before).
We use the command `geemap.Map` with the default Google maps as a base layer. 

Then, we add our SAR images (2 separate layers) using the command ".addLayer" twice. 
- 1st argument represents the variable we want to add (new RGB images in this case).
- 2nd argument represents the visualization parameters (different values for VV and VH RGB images).
- 3rd argument is the name of the layer to be displayed on the map window.


In [18]:
# Create RGB images
S1_VV_RGB = ee.Image.cat(pre_event_filtered.select('VV'), post_event_filtered.select('VV'), pre_event_filtered.select('VV'))
S1_VH_RGB = ee.Image.cat(pre_event_filtered.select('VH'), post_event_filtered.select('VH'), pre_event_filtered.select('VH'))

# Add images to "Layers" in order to visualize them
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(S1_VV_RGB, {'min':-18, 'max':0}, 'S1-VV-RGB')
Map.addLayer(S1_VH_RGB, {'min':-25, 'max':-5}, 'S1-VH-RGB')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Compare before and after hurricane images
We compute the ratio between the before and after images


### Load a map and add our results 

First, we specify the center of the map coordinates and the zoom level (same step as before).
We use the command `geemap.Map` with the default Google maps as a base layer. 

Then, we add our SAR difference image layer using the command ".addLayer". 
- 1st argument represents the variable we want to add (new difference image in this case).
- 2nd argument represents the visualization parameters (min & max values).
- 3rd argument is the name of the layer to be displayed on the map window.


## Apply a threshold to the SAR difference image 
This is also a common practice in image analysis. This way, we separate flooded areas and produce a binary image (1 and 0 values). These values represent SAR difference within our threshold interval or outside our threshold interval. We use the command `.gt`  which stands for "Greater-Than" and the command `.lt`  which stands for "Less-Than".


### Load a map and add our results

Because we are loading this result in the same map, we do NOT specify center coordinates and zoom again.

Then, we add our SAR difference image layer using the command ".addLayer". 
- 1st argument represents the variable we want to add (new difference image in this case).
Here, we used the binary inundation image as a mask ".updateMask" to display only the values = 1
- 2nd argument represents the visualization parameters (color palette "Green-Blue" or GnBu in this case).
- 3rd argument is the name of the layer to be displayed on the map window.

In [19]:
# Compute ratio between before and after images
differenceVH = post_event_filtered.select('VH').divide(pre_event_filtered.select('VH'))

# Add images to "Layers" in order to visualize them
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(differenceVH, {'min': 0, 'max':2}, 'difference VH filtered')

# Apply a threshold - based on difference image values
UPPER_THRESHOLD = 1.15
LOWER_THRESHOLD = 0.7
inundation1 = differenceVH.gt(UPPER_THRESHOLD).Or(differenceVH.lt(LOWER_THRESHOLD))

# Add images to "Layers" in order to visualize them
Map.addLayer(inundation1.updateMask(inundation1),
             {'palette':"GnBu"},'Flooded Areas - RAW')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Calculate pixel connectivity and remove isolated pixels
Isolated pixels are most likely outliers that should be removed. We do this in two steps:

- Calculate connected pixels count using the command ".connectedPixelCount".
This is applied to the original inundation image.
- Generate a new binary innundation image that excludes small patches from the previous inundation image.
This is done by excluding pixels that are connected to less than 8 other pixels.  
For this, we use the command ".updateMask" and combine it with the filter ".gte" (Greater-Than-or-Equal) 


### Load a new blank map and add our results
Here, we display the study area polygon to have better contrast when visualizing the flooded area results.

- First, we set the map center coordinates and zoom level (same step as before).
- Add "roi" layer (color gray)
- Add the new "inundation" layer with no isolated pixels (color palette "GnBu"). 
The new binary inundation image is used as a mask with the command `.updateMask` 


In [20]:
# Calculate pixel connectivity and remove those connected by less than 8 pixels.
connections = inundation1.connectedPixelCount()
inundation2 = inundation1.updateMask(connections.gte(8))

# Add images to "Layers" in order to visualize them
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(roi, {'color': 'gray'}, 'Study Area')
Map.addLayer(inundation2.updateMask(inundation2), {'palette':"GnBu"},'Flooded Areas - non-sparse')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Remove misclassified pixel where the slope is higher than 5%
Water collection in slopes higher than 5% is unlikely since it flows downhill, and means that these inundated pixels are most likely misclassified and should be removed. To do this, we need to load a new dataset: **Digital Elevation Model (DEM)** and compute the slope. Then we can use the slope to mask out areas with values higher than 5%.

__-->__ ***USGS/SRTMGL1_003*** global elevation raster image is part of the GEE collection. More information can be found here: https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003?hl=en

- Load the global elevation data using the command "ee.Image"
- Use the GEE algorithms "terrain to compute slope, aspect, and hillshade using the DEM

__-->__ ` ee.Algorithms`  is a collection of image analysis and data processing functions. These can be general mathematical functions or functions that are specific product (satellite image or geospatial dataset). More information about the specific **terrain** algorithm can be found here: https://developers.google.com/earth-engine/apidocs/ee-algorithms-terrain?hl=en

- Select the "slope" layer from the terrain results using the command `.select`
- Generate a new binary innundation image that excludes slopes > 5%.
This is done using the command ".updateMask" combined with the command `.lt` which stands for Less-Than. 


### Load a new blank map and add our results
Here, we display the study area polygon to have better contrast when visualizing the flooded area results.

- First, we set the map center coordinates and zoom level (same step as before).
- Add the "DEM" layer (specific min/max values)
- Add "roi" layer (color gray)
- Add the new "inundation-slope-corrected" layer (color palette `GnBu` (Green-Blue))
The new binary inundation image is used as a mask with the command `.updateMask` 


In [21]:
# Remove misclassified pixels in areas with slopes greater than 5%
srtm = ee.Image('USGS/SRTMGL1_003')
terrain = ee.Algorithms.Terrain(srtm)
slope = terrain.select('slope')
inundation3 = inundation2.updateMask(slope.lt(5))

# Add images to "Layers" in order to visualize them
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(srtm, {'min':0, 'max':1000}, 'SRTM')
Map.addLayer(roi, {'color': 'gray'}, 'Study Area')
Map.addLayer(inundation3.updateMask(inundation3),{'palette':"GnBu"},'Flooded Areas - slope-adjusted')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

- **Calculate inundation extent** - First, we multiply the latest binary inundation image by the pixel area value. For this, we use the command `.multiply`  and combine it with the function `ee.Image.pixelArea`  (described below).

__-->__ ` ee.Image.pixelArea`  is function that Generate an image in which the value of each pixel is the area of that pixel in square meters. The returned image has a single band called **area** (https://developers.google.com/earth-engine/apidocs/ee-image-pixelarea?hl=en).


- __Calculate the total area of inundated pixels -__ Here, we use the function `.reduceRegion`  to compute the sum (area) of all the inundated pixels.

__-->__ ` ee.Image.reduceRegion`  is function that Apply a reducer to all the pixels in a specific region. A reducer is a statistical metric such as **mean, median, standard deviation, etc.** This function allows for several other options which can be seen using this link: https://developers.google.com/earth-engine/apidocs/ee-image-reduceregion?hl=en

The options used in this example are:

- `reducer = ee.Reducer.sum()` --> to select the Sum statistics 
- `geometry = roi` --> to limit the extent to our study area
- `scale = 10` --> which corresponds to our SAR image native spatial resolution
- `maxPixels = 1e9` --> this option is commented out. 
It is used to request that GEE processess a larger number of pixels.
The request is not always possible to do because GEE has some limits on processing power and request size.
- `bestEffort = True` --> is set to reduce the processing time.


In [22]:
# Calculate inundation extent. Create a raster that contains information on pixel area
inundation_area_pixel = inundation3.multiply(ee.Image.pixelArea())

# Sum the area covered by inundated pixels
# 'bestEffort: True' to reduce processing time. Note - for more accurate results set
# bestEffort to False and increase 'maxPixels'.
inundation_stats = inundation_area_pixel.reduceRegion(
                                         reducer= ee.Reducer.sum(),
                                         geometry= roi,
                                         scale= 10, # native resolution
                                        #  maxPixels= 1e9,
                                         bestEffort= True)


## Convert inundated area to hectares
We use the `inundated_stats`  variable (type dictionary) from the previous step. 

__-->__ The function `.getNumber`  may be applied to different types of datasets/variables as can be seen here https://developers.google.com/s/results/earth-engine?hl=en&q=getNumber

The function `Dictionary.getNumber`  is used in this case requires one argument, which is the `key`  (value "VH").

Then, we use the function `.divide`  and the value 10000, which like the name indicates divides the input value (inundation_stats), which is in square meters, by 10000 to get the result in hectares.

Finally, we use the function `.round`  to round up the result in hectares.


### Print the results to display
Here, we simply use the python `print`  command. To obtain the actual value (hectares), we must use the GEE command `.getInfo()`  as seen below. Additionally, the result is an integer that must be converted to a python string for printing using the command ` str()` 


In [23]:
# Convert inundated extent to hectares
inundation_area_ha = inundation_stats \
                          .getNumber("VH") \
                          .divide(10000) \
                          .round()

# print the results to display
print(f'Calculation result --> Estimated flood extent from Hurricane Maria:\n {inundation_area_ha.getInfo()} Hectares')

Calculation result --> Estimated flood extent from Hurricane Maria:
 33121 Hectares


# Analysis of Multispectral imaging from Sentinel-2
## Create a function for cloud masking specific to Sentinel-2 images
This function is simply converted from Javascript syntax to python. It already exists on GEE documentation here https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2?hl=en#bands

In summary, we can use the **pixel quality (QA) band** of a Sentinel-2 image to identify **cloudy pixels** and mask them out. (The QA band varies from one satellite to another and a different function would be needed for each one). For Sentinel-2, we use the band ` QA10`  and the data-bits 10 and 11 to identify clouds and cirrus pixels. We create a mask by using the command `.bitwiseAnd`  and the filter `.eq`  to identify pixels where the data bits 10 and 11 are equal to Zero. Finally, the cloud mask is applied to the input image of the function (which would be a Sentinel-2 image) using the command `.updateMask`  and the result is divided by 10000 to rescale the reflectance values.

### Set visualization parameters for S2 imagery
This is done by creating a dictionary with keys representing different parameters and the values of each key is assigned by the user. In this example we define the following map visualization parameters (specifically for Sentinel-2 images:
     
- The selected bands are B4, B3, and B2 (represent Red, Green, and Blue)
- The minimum and maximum reflectance values of 0, and 0.4 (two different parameters)
- Gamma values of 0.98, 1.1 and 1 for the respective bands


In [24]:
# Function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
            .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

# Set visualization parameters for S2 imagery
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
  'gamma': [0.98, 1.1, 1]
}

## Load post-hurricane Sentinel-2 Top-Of-Atmosphere (TOA) imagery
We do this separately before the event and then after the event for our study area. 

__-->__ ***The COPERNICUS/S1_GRD*** are ***satellite images from Sentinel-2*** that are part of the GEE collection. More information can be found here: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2?hl=en#description

This dataset contains historical satellite images since 2015 (ongoing satellite mission). It can be loaded using the command `ee.ImageCollection`. Then, we apply multiple filters using the command `.filter`  to select a specific cloud percentage threshold, dates, and boundary. More specifically:

The command `ee.Filter.lt` allows us to select images with cloud cover less-than 10%

The command `.filterDate` allows us to extract available satellite images between a start anad end data.
Here, we used `2017-10-01 to 2018-10-31` time series to minimize data gap caused by cloudy pixels.

The command `.filterBounds` allows us to clip the image to our "roi" study area

The command `.map` allows us to apply the cloud masking function (defined above) to remove cloudy pixels


### Calculate the median pixel value from the Sentinel-2 time series 
This is done using the command `.median()`  applied to the time series variable. Next, we clip the median image using the `.clip`  command and our study area polygon variable ` (roi)`  that was created in a previous step. 


### Load a new blank map and add our results

- First, we set the map center coordinates and zoom level (same step as before).
- Add the "median image" layer using the visualization parameters defined in the previous step.


In [25]:
# Create a variable for post-hurricane Sentinel-2 TOA imagery
# and filter by cloud cover (<10%), date, roi, and apply cloud mask
post_Maria = ee.ImageCollection('COPERNICUS/S2') \
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
               .filterDate('2017-10-01', '2018-10-31') \
               .filterBounds(roi) \
               .map(maskS2clouds)

# Take median pixel value from time series and add as a Layer
medianpixels1 = post_Maria.median()
medianpixelsclipped1 = medianpixels1.clip(roi)
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(medianpixelsclipped1, vizParams, 'Post-Maria S2')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

- **Repeat the previous step to load pre-hurricane Sentinel-2 Top-Of-Atmosphere (TOA) imagery** - This time, we use a different dates interval for the time series: '2015-08-31', '2017-08-31'. The only other change is the variable names.

In [26]:
# Create a variable for pre-hurricane Sentinel-2 TOA imagery
# and filter by cloud cover (<10%), date, roi, and apply cloud mask
pre_Maria = ee.ImageCollection('COPERNICUS/S2') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
              .filterDate('2015-08-31', '2017-08-31') \
              .filterBounds(roi) \
              .map(maskS2clouds)

# Take median pixel value from time series and add as a Layer
medianpixels2 = pre_Maria.median()
medianpixelsclipped2 = medianpixels2.clip(roi)
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(medianpixelsclipped2, vizParams, 'Pre-Maria S2')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Calculate post-hurricane NDVI image
The formula for NDVI (Normalized Difference Vegetation Index) is: 

$$\frac{(NIR - Red)}{(NIR + Red)}$$

This is done using the command `.normalizedDifference`  and the bands ` B8`  (Near InfraRed or NIR) and ` B4`  (Red) as input. More information on this function can be found here https://developers.google.com/earth-engine/apidocs/ee-image-normalizeddifference?hl=en

__--> What is an index (in Remote Sensing)?__ An index is a formula computed with the data from multi-spectral imagery bands. The values of these indices can be used to identify different types of material. NDVI is a common one used for identifying vegetation. For a list of common indices used, see the [index database](https://www.indexdatabase.de/). You can see all of the indexes that can be calculated from Sentinel-2A data in [this page](https://www.indexdatabase.de/db/is.php?sensor_id=96)

- **Set visualization parameters for NDVI images** - This is done by creating a dictionary with keys representing different parameters and the values of each key is assigned by the user. In this example we define the following map visualization parameters (specifically for Sentinel-2 images:

      - The minimum and maximum reflectance values of -0.2, and 0.8 (two different parameters)
      - Select a custom color palette with ['blue', 'white', 'yellow', 'green']

- **clip the NDVI image** using the `.clip`  command and our study area polygon variable ` (roi)`  that was created in a previous step. 


- __Load a new blank map and add our results__

      - First, we set the map center coordinates and zoom level (same step as before).
      - Add the "NDVI" layer using the new visualization parameters defined in this step.


In [27]:
# Create post-hurricane NDVI
post_ndvi = medianpixels1.normalizedDifference(['B8', 'B4'])

# Set visualization parameters for NDVI
visParams_ndvi = {'min': -0.2, 'max': 0.8, 'palette': ['blue', 'white', 'yellow', 'green']}

# Clip post-hurricane NDVI to roi
post_ndvi_clip = post_ndvi.clip(roi)

# Add post-hurricane NDVI as a Layer
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(post_ndvi_clip, visParams_ndvi, 'Post-Maria NDVI')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

### Repeat the previous step to create pre-hurricane NDVI image
This time, we use the pre-hurricane time series median image as input. The only other change is the variable names. The same visualization parameters defined in the previous step are used.

In [28]:
# Create pre-hurricane NDVI
pre_ndvi = medianpixels2.normalizedDifference(['B8', 'B4'])
pre_ndvi_clip = pre_ndvi.clip(roi)

# Add pre-hurricane NDVI as a Layer
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(pre_ndvi_clip, visParams_ndvi, 'Pre-Maria NDVI')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Manually computing the index
__-->__ The **normalized difference** form $\frac{x-y}{x+y}$ is common enough that `ee` provides a function for it. However, you can also manually compute an index using the `.add, .subtract, .multiply` and `.divide` functions with the respective bands. This is useful when calculating more complex index formulae. Here we show how to manually compute the **NDVI**, and we can see that we get the same result.

In [29]:
manual_post_ndvi = medianpixels1.select('B8')\
                                .subtract(medianpixels1.select('B4'))\
                                .divide(medianpixels1.select('B8')\
                                        .add(medianpixels1.select('B4'))).clip(roi)

Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(manual_post_ndvi, visParams_ndvi, 'Post-Maria NDVI - manual computation')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Calculate NDVI difference
We use the command `.subtract`  applied to the pre-hurricane NDVI image and subtract from it the post-hurricane NDVI image. 


### Load a new blank map and add our results

- First, we set the map center coordinates and zoom level (same step as before).
- Add the "NDVI difference" layer using visualization parameters specifically defined in this step.
  * min, max values of -0.2 and 0.2; everything below -0.2 is mapped to the min value, and everything above 0.2 is mapped to the max
  * a custom color palette with ['red', 'yellow', 'green']: min value is visualized as red, max is as gree, and linearly interpolated in between


In [30]:
# Create NDVI Difference Map
NDVI_diff = post_ndvi_clip.subtract(pre_ndvi_clip)

# Add NDVI difference as a Layer
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(NDVI_diff, {'min': -0.2, 'max': 0.2, 'palette': ['red', 'yellow', 'green']}, 'NDVI Difference')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Estimating number of people affected by flooding
### Load the JRC Global Human Settlement Popluation Density layer (250m)
This layer provides an estimate of the number of people per cell - This is done using the command ` ee.Image`. Then, we clip the image using our study area polygon ` roi`.


__-->__ **The JRC Global Human Settlement Popluation Density layer** is part of the GEE collection. More information can be found here: https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2016_POP_GPW_GLOBE_V1?hl=en


### Exclude population outside non-flooded areas
Here, we use the command `.updateMask` with our inundation mask to exclude non-flooded pixels.


### Calculate the total population affected by inundation
Here, we use the function `.reduceRegion`  to compute the sum (area) of all the inundated pixels. The options used in this example are:

- `reducer = ee.Reducer.sum()` --> to select the Sum statistics 
- `geometry = roi` --> to limit the extent to our study area
- `scale = 250` --> which corresponds to the population layer spatial resolution in meters
- `maxPixels = 1e9` --> is the maximum number of pixels to consider in this GEE request

### print the results to display
Here, we simply use the python print command. To obtain the actual value (population), we must use the GEE command `.getInfo()` as seen below.

In [31]:
# Import JRC Global Human Settlement Popluation Density layer (250m) - number of people per cell
population_count = ee.Image('JRC/GHSL/P2016/POP_GPW_GLOBE_V1/2015').clip(roi)

# Create a raster showing exposed population only using the resampled flood layer
population_exposed = population_count.updateMask(inundation3).updateMask(population_count)

# Sum pixel values of exposed population raster
stats = population_exposed.reduceRegion(reducer= ee.Reducer.sum(),
                                        geometry= roi,
                                        scale= 250,
                                        maxPixels=1e9)

# Get number of exposed people as integer
number_pop_exposed = stats.getNumber('population_count').round()
print(f'Calculation result --> Estimated Exposed population from Hurricane Maria:\n{number_pop_exposed.getInfo()}')

Calculation result --> Estimated Exposed population from Hurricane Maria:
10970


## Estimate effect of flooding on land usage type
### Load the Copernicus Global Land Service (CGLS) land cover (100m)
This is done using the command `ee.Image`. Then, we select the band `discrete classification` (which provides integer values representing the different land cover categories). Finally, we clip the image using our study area polygon `roi`.


__-->__ **The Copernicus Global Land Service (CGLS) land cover (100m) layer** is part of the GEE collection. More information can be found here: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global?hl=en


### Load a new blank map and add our results
- First, we set the map center coordinates and zoom level (same step as before).
- Add the "NDVI difference" layer using the default visualization parameters "empty dictionary {}"


In [32]:
# Create a variable for Copernicus Global Land Service (CGLS) land cover (100m)
# Clip land cover to roi and add as a Layer
lc = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019") \
       .select('discrete_classification').clip(roi)

# Add pre-hurricane NDVI as a Layer
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(lc, {}, "Land Cover")
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

To get a sense of what each classification represents, you can get their `properties` using the `.get` function. Here we construct a dictionary of the class id's and their descriptions from the dataset's [various properties](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global?hl=en#image-properties)

In [33]:
lc_class_dict = {class_id: label for class_id, label in zip(lc.get('discrete_classification_class_values').getInfo(),
                                                  lc.get('discrete_classification_class_names').getInfo())}
lc_class_dict

{0: 'Unknown. No or not enough satellite data available.',
 20: 'Shrubs. Woody perennial plants with persistent and woody stems\nand without any defined main stem being less than 5 m tall. The\nshrub foliage can be either evergreen or deciduous.\n',
 30: 'Herbaceous vegetation. Plants without persistent stem or shoots above ground\nand lacking definite firm structure. Tree and shrub cover is less\nthan 10 %.\n',
 40: 'Cultivated and managed vegetation / agriculture. Lands covered with temporary crops followed by harvest\nand a bare soil period (e.g., single and multiple cropping systems).\nNote that perennial woody crops will be classified as the appropriate\nforest or shrub land cover type.\n',
 50: 'Urban / built up. Land covered by buildings and other man-made structures.',
 60: 'Bare / sparse vegetation. Lands with exposed soil, sand, or rocks and never has\nmore than 10 % vegetated cover during any time of the year.\n',
 70: 'Snow and ice. Lands under snow or ice cover throughout 

## Extract cropland from the land cover map
First, we generate a binary mask with the land cover value of 40, which corresponds to cropland (Cultivated and managed vegetation / agriculture). This is done using the command `.eq`  to apply the mask "equal-to". Then we apply this mask to the land cover layer itself using the command `.updateMask`  similar to previous steps.


### Extract the affected cropland 
Here, we use the command `.updateMask`  applied to the latest inundation variable (slope-corrected, removed isolated pixels) to get only cropland pixels that were affected by inundation.


### Calculate the affected cropland extent
First, we multiply the affected cropland image by the pixel area value. For this, we use the command `.multiply`  and combine it with the function ` ee.Image.pixelArea`.


### Calculate the total area of affected cropland pixels
Here, we use the function `.reduceRegion`  to compute the sum (area) of all the inundated cropland pixels. The options used in this example are:

- `reducer = ee.Reducer.sum()` --> to select the Sum statistics 
- `geometry = roi` --> to limit the extent to our study area
- `scale = 100` --> which corresponds to the native spatial resolution of the land cover image
- `maxPixels = 1e9` --> is the maximum number of pixels in this GEE request


### Convert inundated area to hectares
We use the `crop_stats`  variable (dictionary) and the function `.getNumber`  to extract the `key`  value "VH". Then, we use the function `.divide`  and the value 10000 to convert the result from square meters to hectares. Finally, we use the function `.round`  to round up the result in hectares.


- __print the results to display__ - Here, we simply use the python print command. To obtain the actual value (hectares), we must use the GEE command .getInfo() as seen below.

In [34]:
# Extract only cropland pixels from CGLS using class value equal to 40
# (e.g., cultivated and managed vegetation/agriculture)
cropmask = lc.eq(40)
cropland = lc.updateMask(cropmask)

# Extract the affected cropland using the flood layer
cropland_affected = inundation3.updateMask(cropland)

# Get pixel area of affected cropland layer
crop_pixelarea = cropland_affected.multiply(ee.Image.pixelArea());

# Sum pixels of affected cropland layer
crop_stats = crop_pixelarea.reduceRegion(
                              reducer = ee.Reducer.sum(), #sum all pixels with area information
                              geometry = roi,
                              scale= 100,
                              maxPixels= 1e9)

# Convert area to hectares
crop_area_ha = crop_stats \
                  .getNumber("VH") \
                  .divide(10000) \
                  .round()

# Print results to Console
print(f'Calculation result --> Estimated Inundated Cropland area from Hurricane Maria:\n{crop_area_ha.getInfo()} Hectares')
#print (crop_area_ha, 'Hectares of Inundated Cropland')

Calculation result --> Estimated Inundated Cropland area from Hurricane Maria:
5686 Hectares


## Visualize before/after
### Set map visualization parameters
This is done for the cropland layers before and after the hurricane. The selected Min and Max are the same (0 - 14). The only difference is the color (Yellow: before hurricane, and Red: after hurricane).


### Load a new blank map and add our results 

- First, we set the map center coordinates and zoom level (same step as before).
- Then, we load the study area polygon in Gray color to improve contrast and better see the results.
- Add the two "cropland" layers (before and after hurricane) using their respective visualization parameters.


In [35]:
# Set cropland visualization parameters
croplandVis_before = {'min': 0, 'max': 14.0, 'palette': ['Yellow'],}
croplandVis_after = {'min': 0, 'max': 14.0, 'palette': ['Red'],}

# Add cropland to map
Map = geemap.Map(center=[18.2208, -66.5901], zoom = 9)
Map.addLayer(roi, {'color': 'gray'}, 'Study Area')
Map.addLayer(cropland, croplandVis_before, 'Cropland')

# Add affected cropland to map
Map.addLayer(cropland_affected, croplandVis_after, 'Affected Cropland')
Map

Map(center=[18.2208, -66.5901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Save a file from GEE
If you want to have a copy of the file, you can save it to a `.tiff` to open in another program, such as QGIS or in python with `rasterio`.

We can use the `geemap.ee_export_image` [function](https://geemap.org/notebooks/11_export_image/#download-an-eeimage) to do so.

In [36]:
geemap.ee_export_image(cropland_affected, filename='affected_cropland.tif', scale=100, region=roi.geometry())

Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/affected_cropland.tif


# Challenge

Using this script as a template, find another major event and examine its impacts. 

- Remember that the DEM, the population layer, and the land cover layer all have **global coverage**. 

- Remember that **Sentinel-1** SAR data is only available **since 2014** and that **Sentinel-2** data us only available **since 2015**.